<a href="https://colab.research.google.com/github/alisb2001/AIpython/blob/main/br.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
file_name="archive.zip"

with ZipFile(file_name, 'r') as zip:
   zip.extractall()
   print("done")

In [ ]:
pip install keras


In [ ]:

with ZipFile(file_name, 'r') as zip:
   zip.extractall()
   print("done")

In [ ]:
!file archive.zip


In [ ]:
pip install flask


In [ ]:
from flask import Flask

app = Flask(__name__)

In [ ]:
@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/api/upload', methods=['POST'])
def upload_image():
    return 'Image uploaded and processed'



In [ ]:
from flask import request

@app.route('/api/upload', methods=['POST'])
def uploadimage():
    if 'file' not in request.files:
        return 'No file part'
    file = request.files['file']
    if file.filename == '':
        return 'No selected file'
    # Process the file here
    return 'Image uploaded and processed'

In [ ]:
@app.route('/api/hello', methods=['GET'])
def hello():
    return 'Hello, World!'

@app.route('/api/json', methods=['GET'])
def json_response():
    data = {'message': 'This is a JSON response'}
    return jsonify(data)

In [ ]:
if __name__ == '__main__':
    app.run()

In [ ]:
!pip install -q keras

In [ ]:
import keras

In [ ]:
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input
from tensorflow.keras.preprocessing import image

In [ ]:
train_path='/content/people_segmentation/segmentation/train.txt'
valid_path='/content/people_segmentation/segmentation/val.txt'

In [ ]:
with open(train_path, 'r') as file:
    train_image_paths = [line.strip() for line in file]

with open(valid_path, 'r') as file:
    valid_image_paths = [line.strip() for line in file]


In [ ]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Define hyperparameters
IMAGE_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.0001
BASE_DIR = '/content/people_segmentation'
MODEL_SAVE_PATH = 'background_removal_model.h5'

# Define the U-Net model architecture (customize as needed)
def create_model(input_shape):
    model = Sequential()

    # Encoder
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Decoder
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(UpSampling2D((2, 2)))
    model.add(Conv2D(1, (3, 3), activation='sigmoid', padding='same'))

    return model

# Load and preprocess the dataset
def load_dataset(base_dir):
    # Load images and masks, resize to IMAGE_SIZE, and normalize
    # Return X_train, y_train, X_val, y_val

    return X_train, y_train, X_val, y_val

# Split the dataset into training and validation
X_train, y_train, X_val, y_val = load_dataset(BASE_DIR)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Create and compile the model
model = create_model(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


model.save(MODEL_SAVE_PATH)


In [ ]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import jaccard_score


# Predict masks on the validation dataset
y_pred = model.predict(X_val)

# Convert predicted masks to binary masks
y_pred_binary = (y_pred > 0.5).astype(np.uint8)

# Calculate accuracy
accuracy = accuracy_score(y_val.flatten(), y_pred_binary.flatten())
print(f"Accuracy: {accuracy:.4f}")

# Calculate Intersection over Union (IoU)
iou = jaccard_score(y_val.flatten(), y_pred_binary.flatten())
print(f"IoU: {iou:.4f}")


In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model('background_removal_model.h5')  # Update with your model path

# Function to remove background from an input image
def remove_background(input_image_path, output_image_path):
    # Load the input image
    input_image = cv2.imread(input_image_path)

    # Preprocess the input image (resize to match model input size)
    input_image = cv2.resize(input_image, (256, 256))  # Update with your model's input size

    # Normalize the input image (if needed)
    # input_image = input_image / 255.0  # Uncomment if your model expects pixel values in [0, 1] range

    # Make a prediction using the model
    mask = model.predict(np.expand_dims(input_image, axis=0))[0]

    # Convert the mask to a binary mask (you can adjust the threshold as needed)
    mask_binary = (mask > 0.5).astype(np.uint8)

    # Apply the mask to the input image to remove the background
    output_image = input_image * mask_binary[:, :, np.newaxis]

    # Save the output image
    cv2.imwrite(output_image_path, output_image)

if __name__ == "__main__":
    input_image_path = 'input_image.jpg'
    output_image_path = 'output_image.jpg'

    remove_background(input_image_path, output_image_path)

    print("Background removed and saved as", output_image_path)
